# **Preprocessing Workflow**
This notebook focuses on preparing the raw victim list for downstream analysis by loading source files, normalizing key text fields, enriching locations, and exporting a cleaned dataset.

In [1]:
import sys
import json
import unicodedata
import re
from pathlib import Path
from difflib import SequenceMatcher
import pandas as pd

# Ensure project modules are discoverable
project_root = (Path.cwd() / '..').resolve()
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

# Load Metro Manila location reference data
reference_path = project_root / 'data' / 'metro_manila_districts.json'
with reference_path.open(encoding='utf-8') as fh:
    metro_manila_reference = json.load(fh)

def normalize_text(value):
    if not isinstance(value, str):
        return ''
    normalized = unicodedata.normalize('NFKD', value)
    normalized = normalized.encode('ascii', 'ignore').decode('ascii')
    normalized = normalized.lower()
    normalized = re.sub(r'\bsta\.?\b', 'santa', normalized)
    normalized = re.sub(r'\bmla\b', 'manila', normalized)
    normalized = ''.join(ch if ch.isalnum() or ch.isspace() else ' ' for ch in normalized)
    normalized = normalized.replace('brgy', 'barangay')
    normalized = normalized.replace('barangays', 'barangay')
    return ' '.join(normalized.split())

BARANGAY_RANGE_RE = re.compile(r'^(?P<start>\d+)\s*(?:[-–]|to)\s*(?P<end>\d+)$')
CONNECTOR_SPLIT_RE = re.compile(r'\band\b|[,&/]', re.IGNORECASE)

def expand_barangay_entry(entry, fallback_coords):
    results = []
    if isinstance(entry, dict):
        name = (entry.get('name') or '').strip()
        coords = entry.get('coordinates', fallback_coords) or fallback_coords
        if name:
            results.append((name, coords))
        return results
    text = str(entry).strip()
    if not text:
        return results
    working = re.sub(r'(?i)brgys?', 'Barangay', text)
    working = re.sub(r'(?i)barangays?', 'Barangay', working)
    segments = [seg.strip() for seg in CONNECTOR_SPLIT_RE.split(working) if seg.strip()]
    if not segments:
        segments = [working.strip()]
    for segment in segments:
        segment = re.sub(r'(?i)^(barangay|brgy)\s*', '', segment)
        segment = segment.replace('.', ' ')
        segment = ' '.join(segment.split())
        if not segment:
            continue
        range_match = BARANGAY_RANGE_RE.match(segment)
        if range_match:
            start = int(range_match.group('start'))
            end = int(range_match.group('end'))
            if start <= end:
                for code in range(start, end + 1):
                    results.append((f'Barangay {code}', fallback_coords))
            continue
        if not re.match(r'^\d', segment):
            continue
        results.append((f'Barangay {segment.upper()}', fallback_coords))
    return results or [(text, fallback_coords)]

def build_location_lookup(reference):
    def safe_float(container, key):
        value = container.get(key) if isinstance(container, dict) else None
        return float(value) if isinstance(value, (int, float)) else None

    entries = []
    for city_entry in reference:
        city_name = city_entry.get('city', '')
        if not city_name:
            continue
        city_coords = city_entry.get('coordinates', {}) or {}
        city_aliases = {
            normalize_text(city_name),
            normalize_text(f'city of {city_name}'),
            normalize_text(f'{city_name} city')
        }
        entries.append({
            'city': city_name,
            'district': None,
            'barangay': None,
            'latitude': safe_float(city_coords, 'latitude'),
            'longitude': safe_float(city_coords, 'longitude'),
            'type': 'city',
            'aliases': {alias for alias in city_aliases if alias}
        })

        for district in city_entry.get('districts', []):
            district_name = district.get('name', '')
            if not district_name:
                continue
            district_coords = district.get('coordinates', city_coords) or {}
            district_aliases = {
                normalize_text(district_name),
                normalize_text(f'{district_name} {city_name}'),
                normalize_text(f'{city_name} {district_name}'),
            }
            if district.get('barangay_range'):
                district_aliases.add(normalize_text(district['barangay_range']))
            entries.append({
                'city': city_name,
                'district': district_name,
                'barangay': None,
                'latitude': safe_float(district_coords, 'latitude'),
                'longitude': safe_float(district_coords, 'longitude'),
                'type': 'district',
                'aliases': {alias for alias in district_aliases if alias},
            })

            barangay_sources = []
            if isinstance(district.get('barangays'), list):
                barangay_sources.extend(district['barangays'])
            if isinstance(district.get('barangay_list'), list):
                barangay_sources.extend(district['barangay_list'])

            seen_barangays = set()
            for barangay_entry in barangay_sources:
                for barangay_name, barangay_coords in expand_barangay_entry(barangay_entry, district_coords):
                    canonical_name = barangay_name.strip()
                    if not canonical_name:
                        continue
                    key = canonical_name.lower()
                    if key in seen_barangays:
                        continue
                    seen_barangays.add(key)
                    alias_candidates = {
                        normalize_text(canonical_name),
                        normalize_text(f'{canonical_name} {district_name}'),
                        normalize_text(f'{canonical_name} {city_name}'),
                    }
                    number_token = canonical_name.replace('Barangay', '').strip()
                    if number_token:
                        alias_candidates.update({
                            normalize_text(f'barangay {number_token}'),
                            normalize_text(f'brgy {number_token}'),
                            normalize_text(number_token),
                            normalize_text(f'{number_token} {district_name}'),
                            normalize_text(f'{number_token} {city_name}'),
                        })
                    entries.append({
                        'city': city_name,
                        'district': district_name,
                        'barangay': canonical_name,
                        'latitude': safe_float(barangay_coords, 'latitude'),
                        'longitude': safe_float(barangay_coords, 'longitude'),
                        'type': 'barangay',
                        'aliases': {alias for alias in alias_candidates if alias}
                    })

    priority_order = {'barangay': 0, 'district': 1, 'city': 2}
    entries.sort(key=lambda entry: priority_order.get(entry['type'], 3))
    return entries

location_lookup = build_location_lookup(metro_manila_reference)

special_locations = [
    {
        'aliases': {
            'manila arena',
            'rizal memorial coliseum',
            'rizal memorial sports complex'
        },
        'result': {
            'city': 'Manila',
            'district': 'Malate',
            'barangay': None,
            'latitude': 14.562222,
            'longitude': 120.993889,
            'type': 'barangay',
            'match_score': 1.0
        }
    }
 ]

def match_location_metadata(raw_location):
    norm_location = normalize_text(raw_location)
    if not norm_location:
        return None

    for special in special_locations:
        if any(alias in norm_location for alias in special['aliases']):
            return special['result'].copy()

    best_scores = {
        'barangay': (None, 0.0),
        'district': (None, 0.0),
        'city': (None, 0.0)
    }

    for entry in location_lookup:
        entry_type = entry['type']
        for alias in entry['aliases']:
            if not alias:
                continue
            if alias in norm_location:
                score = 1.0
            else:
                score = SequenceMatcher(None, alias, norm_location).ratio()
            stored_entry, stored_score = best_scores.get(entry_type, (None, 0.0))
            if score > stored_score:
                best_scores[entry_type] = (entry, score)

    thresholds = {'barangay': 0.7, 'district': 0.6, 'city': 0.55}
    for entry_type in ('barangay', 'district', 'city'):
        entry, score = best_scores.get(entry_type, (None, 0.0))
        if entry and score >= thresholds[entry_type]:
            result = entry.copy()
            result['match_score'] = round(score, 3)
            return result

    fallback_entry = None
    fallback_score = 0.0
    fallback_threshold = 0.45
    for entry_type in ('district', 'city'):
        entry, score = best_scores.get(entry_type, (None, 0.0))
        if entry and score > fallback_score and score >= fallback_threshold:
            fallback_entry = entry
            fallback_score = score
    if fallback_entry:
        result = fallback_entry.copy()
        result['match_score'] = round(fallback_score, 3)
        return result

    return None

# 1. Load Data
df = pd.read_csv("../dataset/Manila_City_Only_List.csv")

# 2. Missing Value Analysis
# We need to count both standard NaNs and text "na"
missing_counts = {}
for col in df.columns:
    nan_count = df[col].isnull().sum()
    # Check for "na" string if column is text
    if df[col].dtype == 'object':
        na_str_count = df[col].astype(str).str.strip().str.lower().eq('na').sum()
    else:
        na_str_count = 0
    missing_counts[col] = nan_count + na_str_count

df.head(5)

,Person ID,AGE,GENDER,Date Reported Missing,Time Reported Missing,Date Last Seen,Location Last Seen,Longtitude,Latitude,Post URL
0,NaN,59,male,1/14/2020,12:48 PM,12/14/2019,"Malate, Manila",NaN,NaN,https://www.facebook.com/share/p/1Fp5H7uddW/
1,NaN,41,Male,01/24/2020,5:12 PM,01/16/2021,"Sampaloc, Manila",NaN,NaN,https://www.facebook.com/share/p/1CwZW3pbpf/
2,NaN,43,Male,2/9/2020,7:03 PM,NaN,"Tondo, Manila",NaN,NaN,https://www.facebook.com/share/p/1CoiXoTEjb/
3,NaN,14,Male,2/15/2020,12:19 PM,na,"Binondo, Manila",NaN,NaN,https://www.facebook.com/share/p/17Umn23xj9/
4,NaN,16,male,03/23/2020,12:25,03/11/2025,"Paco,. Manila",NaN,NaN,https://www.facebook.com/share/p/1BhMzYvEJN/


### Import & Missing Value Audit
- Loads the Metro Manila reference, builds a lookup, and defines the normalization helper.
- Reads the raw CSV and tallies missing markers (`NaN` and literal `"na"`).
- Shows the head of the dataset so we can sanity-check columns before cleaning.

In [2]:
# Calculate total percentage empty
total_cells = df.size
total_missing = sum(missing_counts.values())
percent_empty = (total_missing / total_cells) * 100

print(f"Total Dataset Empty: {percent_empty:.2f}%")
print("Missing Counts per Column:")

missing_df = pd.DataFrame(list(missing_counts.items()), columns=['Column', 'Missing Count'])
missing_df.sort_values(by='Missing Count', ascending=False)

Total Dataset Empty: 35.31%
Missing Counts per Column:


,Column,Missing Count
0,Person ID,113
7,Longtitude,113
8,Latitude,113
5,Date Last Seen,54
4,Time Reported Missing,5
9,Post URL,1
1,AGE,0
2,GENDER,0
3,Date Reported Missing,0
6,Location Last Seen,0


## **Data Preprocessing**
We streamline columns, standardize categorical fields, and convert date/time text into structured formats so downstream analyses operate on clean data.

In [3]:
# 1. Drop Irrelevant Columns
df_clean = df.drop(columns=['Latitude', 'Longtitude'])

In [4]:
# 2. Standardize Demographics
# Clean Gender (remove spaces, capitalize)
df_clean['GENDER'] = df_clean['GENDER'].str.strip().str.capitalize()

In [5]:
# 3. Parse Dates & Times
# Convert Date columns
df_clean['Date Reported Missing'] = pd.to_datetime(df_clean['Date Reported Missing'], errors='coerce')

In [6]:
# Handle 'na' in Date Last Seen before converting
df_clean['Date Last Seen'] = df_clean['Date Last Seen'].replace('na', pd.NA)
df_clean['Date Last Seen'] = pd.to_datetime(df_clean['Date Last Seen'], errors='coerce')

# Clean Time: Handle 'na' and convert mixed formats (12h/24h)
df_clean['Time Reported Missing'] = df_clean['Time Reported Missing'].replace('na', pd.NA)
# The datetime converter infers the correct format automatically
df_clean['Time_Obj'] = pd.to_datetime(df_clean['Time Reported Missing'], errors='coerce')

/var/folders/4d/c9fb2t_j5znf_ftgytncz11h0000gp/T/ipykernel_6673/1347800615.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_clean['Time_Obj'] = pd.to_datetime(df_clean['Time Reported Missing'], errors='coerce')


In [7]:
df_clean.head(5)

,Person ID,AGE,GENDER,Date Reported Missing,Time Reported Missing,Date Last Seen,Location Last Seen,Post URL,Time_Obj
0,NaN,59,Male,2020-01-14,12:48 PM,2019-12-14,"Malate, Manila",https://www.facebook.com/share/p/1Fp5H7uddW/,2025-12-11 12:48:00
1,NaN,41,Male,2020-01-24,5:12 PM,2021-01-16,"Sampaloc, Manila",https://www.facebook.com/share/p/1CwZW3pbpf/,2025-12-11 17:12:00
2,NaN,43,Male,2020-02-09,7:03 PM,NaT,"Tondo, Manila",https://www.facebook.com/share/p/1CoiXoTEjb/,2025-12-11 19:03:00
3,NaN,14,Male,2020-02-15,12:19 PM,NaT,"Binondo, Manila",https://www.facebook.com/share/p/17Umn23xj9/,2025-12-11 12:19:00
4,NaN,16,Male,2020-03-23,12:25,2025-03-11,"Paco,. Manila",https://www.facebook.com/share/p/1BhMzYvEJN/,2025-12-11 12:25:00


## **Age Binning**
Custom logic clusters numeric ages and free-text descriptors into consistent life-stage buckets for clearer demographic charts.

In [8]:
def smart_bin_age(val):
    s_val = str(val).lower().strip()
    if s_val == 'adult': return 'Adult'
    elif s_val in ['senior', 'adult older']: return 'Old'
    elif s_val == 'teen': return 'Teen'
    elif s_val == 'young teen': return 'Young Teen'
    elif s_val == 'young adult': return 'Young Adult'
    elif s_val == 'child': return 'Child'

    try:
        age_num = float(val)
        if pd.isna(age_num): return "Unknown"
        if age_num < 13: return "Child"
        elif 13 <= age_num <= 14: return "Young Teen"
        elif 15 <= age_num <= 17: return "Teen"
        elif 18 <= age_num <= 24: return "Young Adult"
        elif 25 <= age_num <= 59: return "Adult"
        else: return "Old"
    except:
        return "Unknown"

In [9]:
# Apply Logic
df_clean['Age_Group'] = df_clean['AGE'].apply(smart_bin_age)
df_clean['Age_Group'].value_counts(dropna=False)

Age_Group
Adult          50
Old            22
Young Teen     11
Young Adult    11
Teen           10
Child           9
Name: count, dtype: int64

## **City - Barangay Segregation**
Using the JSON lookup we match each record to a Metro Manila city or district, capture coordinates, and flag the confidence level for mapping.

In [10]:
# Derive Metro Manila city/district metadata & coordinates
location_matches = df_clean['Location Last Seen'].apply(match_location_metadata)

def format_city_label(match):
    if not match:
        return "Other/Unknown"
    city_name = match['city'].strip().lower()
    return "Manila City" if city_name == 'manila' else match['city']

df_clean['City_Cleaned'] = location_matches.apply(format_city_label)
df_clean['Barangay_Cleaned'] = location_matches.apply(
    lambda match: match['barangay'] if match and match.get('barangay') else pd.NA
)
df_clean['District_Cleaned'] = location_matches.apply(
    lambda match: match['district'] if match and match.get('district') else pd.NA
)
df_clean['Latitude'] = location_matches.apply(
    lambda match: match['latitude'] if match and match.get('latitude') is not None else pd.NA
)
df_clean['Longitude'] = location_matches.apply(
    lambda match: match['longitude'] if match and match.get('longitude') is not None else pd.NA
)
df_clean['Location_Match_Level'] = location_matches.apply(
    lambda match: match['type'] if match else "none"
)
df_clean['Location_Match_Score'] = location_matches.apply(
    lambda match: match['match_score'] if match and match.get('match_score') is not None else pd.NA
)

# Filter to Manila City only to drop other locations
df_clean = df_clean[df_clean['City_Cleaned'] == "Manila City"].copy()

# 3. Clean Date & Time
df_clean['Date Reported Missing'] = pd.to_datetime(df['Date Reported Missing'], errors='coerce')
df_clean['Year'] = df_clean['Date Reported Missing'].dt.year

df_clean['Time Reported Missing'] = df_clean['Time Reported Missing'].replace('na', pd.NA)
df_clean['Time_Obj'] = pd.to_datetime(df_clean['Time Reported Missing'], errors='coerce')
df_clean['Hour_Missing'] = df_clean['Time_Obj'].dt.hour

# Ensure deterministic IDs before export
df_clean = df_clean.reset_index(drop=True)
df_clean.insert(0, 'Person_ID', [f"MP-{i:04d}" for i in range(1, len(df_clean) + 1)])

export_columns = [col for col in df_clean.columns if True]
df_clean.to_csv("Missing People - cleaned.csv", index=False, columns=export_columns)

del location_matches

/var/folders/4d/c9fb2t_j5znf_ftgytncz11h0000gp/T/ipykernel_6673/3754816746.py:38: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_clean['Time_Obj'] = pd.to_datetime(df_clean['Time Reported Missing'], errors='coerce')


In [11]:
df_clean.head(5)

,Person_ID,Person ID,AGE,GENDER,Date Reported Missing,Time Reported Missing,Date Last Seen,Location Last Seen,Post URL,Time_Obj,Age_Group,City_Cleaned,Barangay_Cleaned,District_Cleaned,Latitude,Longitude,Location_Match_Level,Location_Match_Score,Year,Hour_Missing
0,MP-0001,NaN,59,Male,2020-01-14,12:48 PM,2019-12-14,"Malate, Manila",https://www.facebook.com/share/p/1Fp5H7uddW/,2025-12-11 12:48:00,Adult,Manila City,<NA>,Malate,14.5714,120.9904,district,1.0,2020.0,12.0
1,MP-0002,NaN,41,Male,2020-01-24,5:12 PM,2021-01-16,"Sampaloc, Manila",https://www.facebook.com/share/p/1CwZW3pbpf/,2025-12-11 17:12:00,Adult,Manila City,<NA>,Sampaloc,14.6133,121.0003,district,1.0,2020.0,17.0
2,MP-0003,NaN,43,Male,2020-02-09,7:03 PM,NaT,"Tondo, Manila",https://www.facebook.com/share/p/1CoiXoTEjb/,2025-12-11 19:03:00,Adult,Manila City,<NA>,Tondo,14.6186,120.9681,district,1.0,2020.0,19.0
3,MP-0004,NaN,14,Male,2020-02-15,12:19 PM,NaT,"Binondo, Manila",https://www.facebook.com/share/p/17Umn23xj9/,2025-12-11 12:19:00,Young Teen,Manila City,<NA>,Binondo,14.6006,120.9754,district,1.0,2020.0,12.0
4,MP-0005,NaN,16,Male,2020-03-23,12:25,2025-03-11,"Paco,. Manila",https://www.facebook.com/share/p/1BhMzYvEJN/,2025-12-11 12:25:00,Teen,Manila City,<NA>,Paco,14.5833,120.9961,district,1.0,2020.0,12.0
